In [4]:
import os
import cv2
import numpy as np


In [ ]:

data_dir = "Major2-bloodgroup-fingerprint"
data = []
labels = []
classes = ['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-']
for label, blood_group in enumerate(classes):
    path = os.path.join(data_dir, blood_group)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 128))
        data.append(image)
        labels.append(label)

data = np.array(data) / 255.0
labels = np.array(labels)

In [54]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

dataset_path = "Major2-bloodgroup-fingerprint"

classes = ['A+', 'A-', 'AB+', 'AB-', 'B+', 'B-', 'O+', 'O-']

# Define ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalization
    validation_split=0.2  # 80-20 train-validation split
)

# Load data
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize for ResNet50 or VGG16
    batch_size=32,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale',
    classes= classes
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale',
    classes= classes,
)

# Display class indices
print("Class Labels:", train_data.class_indices)

Found 6336 images belonging to 8 classes.
Found 1584 images belonging to 8 classes.
Class Labels: {'A+': 0, 'A-': 1, 'AB+': 2, 'AB-': 3, 'B+': 4, 'B-': 5, 'O+': 6, 'O-': 7}


In [30]:
def clahe_preprocessing(image):
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Ensure the image is 8-bit
    image = image.astype(np.uint8)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=-1)
    return image


In [ ]:
datagen_clahe = ImageDataGenerator(preprocessing_function=clahe_preprocessing, validation_split=0.2)
classes = ['A+', 'A-', 'AB+', 'AB-', 'B+', 'B-', 'O+', 'O-']


train_data_clahe = datagen_clahe.flow_from_directory(
    dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training', color_mode='grayscale', classes=classes
)
val_data_clahe = datagen_clahe.flow_from_directory(
    dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation', color_mode='grayscale', classes=classes
)

Found 6336 images belonging to 8 classes.
Found 1584 images belonging to 8 classes.


In [47]:
print("Class labels detected:", train_data_clahe.class_indices)


Class labels detected: {'A+': 0, 'A-': 1, 'AB+': 2, 'AB-': 3, 'B+': 4, 'B-': 5, 'O+': 6, 'O-': 7}


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/Users/insiyar/Documents/Development/bloodgroup-fingerprint/fingerprint_env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',            # Metric to monitor
    factor=0.5,                    # Factor to reduce the learning rate
    patience=2,                    # Wait 2 epochs before reducing
    min_lr=1e-3                    # Minimum learning rate
)
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',      # File path to save the model
    monitor='val_accuracy',        # Metric to monitor
    save_best_only=True            # Save only the best model
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)


In [ ]:
history_clahe = model.fit(train_data_clahe, validation_data=val_data_clahe, epochs=10, callbacks=[early_stopping])


Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 538ms/step - accuracy: 0.1290 - loss: 14.0386

198/198 ━━━━━━━━━━━━━━━━━━━━ 115s 579ms/step - accuracy: 0.1290 - loss: 14.0390 - val_accuracy: 0.1250 - val_loss: 14.1033 - learning_rate: 0.0010
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 109s 547ms/step - accuracy: 0.1292 - loss: 14.0355 - val_accuracy: 0.1250 - val_loss: 14.1033 - learning_rate: 0.0010
Epoch 3/10
185/198 ━━━━━━━━━━━━━━━━━━━━ 5s 400ms/step - accuracy: 0.1140 - loss: 14.2800

KeyboardInterrupt: 

In [ ]:
def nlm_denoising_preprocessing(image):

    image = cv2.fastNlMeansDenoising(image, None, h=10, templateWindowSize=7, searchWindowSize=21)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=-1)
    return image


In [48]:
def median_blur_preprocessing(image):

    image = cv2.medianBlur(image, 5)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=-1)
    return image


In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

median_datagen = ImageDataGenerator(preprocessing_function=median_blur_preprocessing, validation_split=0.2)
# Create training generator
median_train_data = median_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',  
    subset='training',
    classes=classes
)

# Create validation generator
median_val_data = median_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    subset='validation',
    classes=classes
)

Found 6336 images belonging to 8 classes.
Found 1584 images belonging to 8 classes.


In [50]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

median_history = model.fit(
    median_train_data,
    epochs=10,
    validation_data=median_val_data,
    callbacks=[early_stopping]
)

Epoch 1/10


/Users/insiyar/Documents/Development/bloodgroup-fingerprint/fingerprint_env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


198/198 ━━━━━━━━━━━━━━━━━━━━ 106s 520ms/step - accuracy: 0.1292 - loss: 14.0361 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 139s 701ms/step - accuracy: 0.1233 - loss: 14.1303 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 111s 561ms/step - accuracy: 0.1285 - loss: 14.0463 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 89s 445ms/step - accuracy: 0.1304 - loss: 14.0170 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.


In [52]:
loss, accuracy = model.evaluate(median_val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.1272 - loss: 14.0677
Validation Loss: 14.1033
Validation Accuracy: 0.1250


In [55]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

basic_history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=[early_stopping]
)

Epoch 1/10


/Users/insiyar/Documents/Development/bloodgroup-fingerprint/fingerprint_env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


198/198 ━━━━━━━━━━━━━━━━━━━━ 60s 287ms/step - accuracy: 0.1288 - loss: 14.0427 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 53s 267ms/step - accuracy: 0.1240 - loss: 14.1197 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 49s 249ms/step - accuracy: 0.1222 - loss: 14.1492 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 52s 260ms/step - accuracy: 0.1205 - loss: 14.1763 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 53s 265ms/step - accuracy: 0.1217 - loss: 14.1559 - val_accuracy: 0.1250 - val_loss: 14.1033
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 2.


In [56]:
loss, accuracy = model.evaluate(median_val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.1378 - loss: 13.8968
Validation Loss: 14.1033
Validation Accuracy: 0.1250
